In [6]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../../ml/"))

from dev_wtp_io_utils import *

In [7]:
# nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
# ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

sys.path.append(os.path.abspath("../../ml/"))
from apply_loocv_and_save import load_and_preprocess
from dev_utils import read_yaml_for_host
import warnings


config_data = read_yaml_for_host("sst_config.yml")



In [8]:
nonbids_data_path = config_data['nonbids_data_path']
ml_data_folderpath = nonbids_data_path + "fMRI/ml"


In [9]:
nonbids_data_path

'/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/'

In [10]:
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20211027T173724.csv")
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20220818T144138.csv")
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20230102T164214.csv")


In [11]:
include_exclude_list = pd.read_csv("../nsc_subject_exclusions.csv")

In [12]:
test_train_df_raw = test_train_df_raw.merge(include_exclude_list[include_exclude_list.Task=='SST'],left_on='sub_label',right_on='SubjectId',how='left')
test_train_df_raw.loc[test_train_df_raw.Include.isna(),'Include'] = True
test_train_df = test_train_df_raw[test_train_df_raw.Include==True]

In [13]:
exclude_subjects = [
    'DEV012','DEV018', 'DEV068', 'DEV073','DEV159', #excluded during the conversion process for reasons I'm not sure about

    'DEV061','DEV185','DEV187',
    'DEV189','DEV190','DEV192',
    'DEV198','DEV203','DEV220','DEV221']
#2022-01-06 added DEV157 because of an error
#2023-01-02 to -03 added DEV068, DEV101 because of an error

In [14]:
train_subjs = test_train_df.loc[test_train_df.SplitGroup_75_25=='Train','sub_label'].tolist()#only get the train subjects; ignore those previously marked hold-out

In [15]:
def get_Brain_Data_for_sub(sub_label,behavdesign,betaseries_path='/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/'):
    return(dev_wtp_io_utils.get_Brain_Data_betas_for_sub(sub_label,behavdesign,betaseries_path))

In [17]:
from dev_wtp_io_utils import import_sst_betaseries_w1_subjs_to_masked_numpy_pkl

In [18]:
all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")
#cut out other waves
sst_1_behavdesign_1 = all_sst_events[(all_sst_events.wave=='wave1')].reset_index(drop=True)
#label the ITI as such
sst_1_behavdesign_1.loc[(all_sst_events.trial_type.isna()),'trial_type']='ITI'
#ensure it is correctly ordered
sst_1_behavdesign_clean = sst_1_behavdesign_1.sort_values(['subject','onset']).reset_index(drop=True)
#now tell it the name of the corresponding beta. this is easy; they are all consecutive
sst_1_behavdesign_clean['beta'] = 'beta_' + (sst_1_behavdesign_clean.groupby(['subject','wave']).cumcount()+1).astype(str).apply(lambda x: x.zfill(4)) + '.nii'

In [19]:
#I want to import this other module; we don't actually use it but it's a good reference
#for future programmers to see
#it's important we keep the logic consistent for the two main conditions
sys.path.append(os.path.abspath("../../fx/multiconds/SST/"))
from multiconds import create_posterror_masks_from_masks

#sst_1_behavdesign_clean['prev_trial_type'] = sst_1_behavdesign_clean['trial_type']
sst_1_behavdesign_clean['prev_trial_type'] = (
    sst_1_behavdesign_clean
    .loc[sst_1_behavdesign_clean['trial_type']!='ITI']
    .groupby(['subject','wave'])
    .shift(1)
    ['trial_type']
)#this works because during assignment, items will be assigned to the line corresponding to their index
#very important you don't reset index during this operation.
is_post_stop_correct_go = (
    (sst_1_behavdesign_clean['trial_type']=='correct-go') & 
    (sst_1_behavdesign_clean['prev_trial_type'].isin(['failed-stop','correct-stop']))
)

sst_1_behavdesign_clean.loc[is_post_stop_correct_go,'post_stop_correct_go_subtype'] = 'Other'
sst_1_behavdesign_clean.loc[is_post_stop_correct_go & (sst_1_behavdesign_clean['prev_trial_type']=='failed-stop'),'post_stop_correct_go_subtype'] = 'post-error'
sst_1_behavdesign_clean.loc[is_post_stop_correct_go & (sst_1_behavdesign_clean['prev_trial_type']=='correct-stop'),'post_stop_correct_go_subtype'] = 'post-correct'


In [20]:
sst_1_behavdesign_clean.groupby(['trial_type','post_stop_correct_go_subtype']).size().reset_index()

,trial_type,post_stop_correct_go_subtype,0
0,correct-go,post-correct,1879
1,correct-go,post-error,2145


In [21]:
sst_1_behavdesign_clean['trial_type'].value_counts()

ITI             26112
correct-go      18750
failed-stop      3434
correct-stop     3094
failed-go         834
Name: trial_type, dtype: int64

In [22]:
behavdesign = sst_1_behavdesign_clean
sst_1_behavdesign_clean_dev005 = behavdesign[[s in 'DEV005' for s in behavdesign.subject]]

## Just subjects in the conditions of interest: CorrectStop vs. CorrectGo

In [23]:
sst_1_behavdesign_clean.trial_type.value_counts()

ITI             26112
correct-go      18750
failed-stop      3434
correct-stop     3094
failed-go         834
Name: trial_type, dtype: int64

In [24]:
sst_1_behavdesign_correct_conditions=sst_1_behavdesign_clean[sst_1_behavdesign_clean.trial_type.isin(['correct-go','correct-stop'])]

In [25]:
sst_1_behavdesign_correct_conditions.trial_type.value_counts()

correct-go      18750
correct-stop     3094
Name: trial_type, dtype: int64

In [26]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq


train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

### Now mask those subjects

In [27]:
# get the PFC mask
mask_path = ml_data_folderpath + '/prefrontal_cortex.nii.gz'
mask_pfc = nib.load(ml_data_folderpath + '/prefrontal_cortex.nii.gz')

In [30]:
" ".join(train_subjs)

'DEV011 DEV020 DEV047 DEV063 DEV064 DEV067 DEV075 DEV088 DEV094 DEV097 DEV100 DEV102 DEV105 DEV107 DEV111 DEV113 DEV114 DEV116 DEV118 DEV119 DEV135 DEV138 DEV141 DEV144 DEV145 DEV155 DEV156 DEV157 DEV158 DEV159 DEV161 DEV164 DEV168 DEV169 DEV171 DEV173 DEV174 DEV176 DEV177 DEV178 DEV183 DEV185 DEV186 DEV187 DEV189 DEV191 DEV192 DEV193 DEV194 DEV196 DEV197 DEV198 DEV201 DEV202 DEV203 DEV205 DEV206 DEV208 DEV209 DEV211 DEV215 DEV216 DEV217 DEV218 DEV220 DEV221 DEV222 DEV223 DEV224 DEV004 DEV005 DEV006 DEV009 DEV010 DEV012 DEV013 DEV014 DEV015 DEV016 DEV017 DEV018 DEV019 DEV021 DEV022 DEV023 DEV024 DEV025 DEV026 DEV027 DEV028 DEV029 DEV030 DEV033 DEV034 DEV035 DEV036 DEV038 DEV039 DEV040 DEV041 DEV042 DEV043 DEV044 DEV045 DEV046 DEV048 DEV049 DEV050 DEV051 DEV052 DEV053 DEV054 DEV055 DEV056 DEV057 DEV058 DEV059 DEV060 DEV061 DEV062 DEV065 DEV066 DEV068 DEV069 DEV070 DEV071 DEV073 DEV074 DEV076 DEV077 DEV079 DEV080 DEV083 DEV084 DEV085 DEV086 DEV087 DEV089 DEV210 DEV228 DEV230 DEV231 DEV23

In [32]:
import_sst_betaseries_w1_subjs_to_masked_numpy_pkl(
    train_subjs[0:2],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    mask_path=mask_path,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':mask_pfc
    }
    )

/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/
DEV011
...............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration  trial_type subject   wave           beta  \
1794    2.75834   0.64209  correct-go  DEV011  wave1  beta_0003.nii   
1796    5.51390   0.56936  correct-go  DEV011  wave1  beta_0005.nii   
1798    9.02224   0.46994  correct-go  DEV011  wave1  beta_0007.nii   
1800   12.52780   0.40205  correct-go  DEV011  wave1  beta_0009.nii   
1802   15.90975   0.44627  correct-go  DEV011  wave1  beta_0011.nii   
...         ...       ...         ...     ...    ...            ...   

/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nltools/data/brain_data.py:887: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out.X = self.X.append(data.X, **kwargs)


IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
print("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:6],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':mask_pfc
    }
)

In [ ]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:30],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':True,
        'mask':mask_pfc
    }
)

### Standardize by condition

In [ ]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:2],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_pfc_condstd',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':True,
        'mask':mask_pfc
    }
    )

In [ ]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:6],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_pfc_condstd',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':True,
        'mask':mask_pfc
    }
    )

In [ ]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:30],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_pfc_condstd',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':True,
        'mask':mask_pfc
    }
    )

In [ ]:
train_subjs_selected

In [ ]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs_selected,'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_pfc_condstd',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':True,
        'mask':mask_pfc
    }
)